# Part (a): Simulation Function

In this part, we implement the **forward pass** of the given neural network.

Each node is defined as:

- **Pre-activation:**  
  \[
  a_i = \sum_j w_{ij} z_j
  \]

- **Activation:**  
  \[
  z_i = f_i(a_i)
  \]

with \(z_5 = a_5\) (the output node is linear).

The activation functions are:

- \(f_1(x) = \sigma(x) = \dfrac{1}{1+e^{-x}}\)  
- \(f_2(x) = \text{ReLU}(x) = \max(0, x)\)  
- \(f_3(x) = f_4(x) = \sigma(x)\)  
- \(f_5(x) = \text{Identity}(x) = x\)

---

### Implementation Notes
- The network is represented by a **5×5 weight matrix** \(B\), where  
  \[
  B[i, j] = w_{ij}
  \quad\text{if there is a connection from node \(j\) to node \(i\), otherwise } 0.
  \]
- The `forward_pass(x, weights)` function:
  - Takes scalar input \(x\) at node 1,
  - Computes all \(a_i\) and \(z_i\),
  - Returns a dictionary with the values of `a`, `z`, and `B`.

---

### Example Output
For input \(x = 0.8\), the simulation produced:

In [3]:
import math
from typing import Dict, Tuple, List

Idx = int                # node index in {1,2,3,4,5}
Edge = Tuple[Idx, Idx]   # (to_i, from_j)

def sigmoid(x: float) -> float:
    return 1.0 / (1.0 + math.exp(-x))

def relu(x: float) -> float:
    return x if x > 0.0 else 0.0

def build_B(weights: Dict[Edge, float]) -> List[List[float]]:
    """
    Build the 5x5 weight matrix B with B[i-1][j-1] = w_j^i if edge j->i exists, else 0.
    Nodes are 1-indexed in the math, 0-indexed in the Python lists.
    """
    B = [[0.0 for _ in range(5)] for _ in range(5)]
    for (i, j), w in weights.items():
        B[i-1][j-1] = float(w)
    return B

def forward_pass(x: float, weights: Dict[Edge, float]):
    """
    x is the scalar input fed at node 1: a1 = x, z1 = sigmoid(a1).
    weights maps (to_i, from_j) -> value for each existing edge j -> i.
    Returns:
      - 'a': [a1,...,a5]
      - 'z': [z1,...,z5]
      - 'B': 5x5 weight matrix as a list of lists
    """
    # Build matrix B once (useful for debugging/printing)
    B = build_B(weights)

    a = [0.0]*5
    z = [0.0]*5

    # Node 1 (input then sigmoid)
    a[0] = float(x)
    z[0] = sigmoid(a[0])     # f1 = sigmoid

    # Helper to compute ai = sum_j B[i][j] * z[j]
    def preact(i: int) -> float:
        return sum(B[i][j] * z[j] for j in range(5))

    # Node 2 (ReLU)
    a[1] = preact(1)
    z[1] = relu(a[1])        # f2 = ReLU

    # Node 3 (sigmoid)
    a[2] = preact(2)
    z[2] = sigmoid(a[2])     # f3 = sigmoid

    # Node 4 (sigmoid)
    a[3] = preact(3)
    z[3] = sigmoid(a[3])     # f4 = sigmoid

    # Node 5 (identity output)
    a[4] = preact(4)
    z[4] = a[4]              # z5 = a5

    return {"a": a, "z": z, "B": B}


In [ ]:
weights = {
    (2,1): 0.7,   # a1 -> a2
    (3,1): -0.4,  # a1 -> a3
    (4,1): 0.2,   # a1 -> a4
    (3,2): 0.9,   # a2 -> a3
    (4,2): -1.1,  # a2 -> a4
    (5,2): 0.5,   # a2 -> a5
    (5,3): 1.2,   # a3 -> a5
    (5,4): -0.3,  # a4 -> a5
}

out = forward_pass(0.8, weights)

# Nicely formatted node outputs
for i, (a_val, z_val) in enumerate(zip(out["a"], out["z"]), start=1):
    print(f"a{i} = {a_val:.4f}, z{i} = {z_val:.4f}")

print("\nWeight matrix B:")
for row in out["B"]:
    print(" ".join(f"{val:6.2f}" for val in row))

a1 = 0.8000, z1 = 0.6900
a2 = 0.4830, z2 = 0.4830
a3 = 0.1587, z3 = 0.5396
a4 = -0.3933, z4 = 0.4029
a5 = 0.7681, z5 = 0.7681

Weight matrix B:
  0.00   0.00   0.00   0.00   0.00
  0.70   0.00   0.00   0.00   0.00
 -0.40   0.90   0.00   0.00   0.00
  0.20  -1.10   0.00   0.00   0.00
  0.00   0.50   1.20  -0.30   0.00
